Trial key limit for Chat Cohere: 10 requests per minute

In [55]:
!pip -q install crewai duckduckgo-search cohere
!pip -q install  decouple langchain-community langchain-cohere

!pip install 'crewai[tools]'

# crewai==0.22.5


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [30]:
!py -3.11 -m pip install --upgrade pip

In [1]:
import os
!pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()


[notice] A new release of pip available: 22.3 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


True

## Tool

In [2]:
!pip -q install  decouple langchain-exa exa_py==1.0.7

In [2]:
from exa_py import Exa
from langchain.agents import tool
import os
from typing import List

class ExaSearchTool:
	@tool
	def search(query: str):
		"""Search for a webpage based on the query."""
		return ExaSearchTool._exa().search(f"{query}", use_autoprompt=True, num_results=3)

	@tool
	def find_similar(url: str):
		"""Search for webpages similar to a given URL.
		The url passed in should be a URL returned from `search`.
		"""
		return ExaSearchTool._exa().find_similar(url, num_results=3)

	@tool
	def get_contents(ids: List[str]):
		"""Get the contents of a webpage.
		The ids must be passed in as a list, a list of ids returned from `search`.
		"""
		contents = str(ExaSearchTool._exa().get_contents(ids))
		print(contents)
		contents = contents.split("URL:")
		contents = [content[:1000] for content in contents]
		return "\n\n".join(contents)

	def tools():
		return [ExaSearchTool.search, ExaSearchTool.find_similar, ExaSearchTool.get_contents]

	def _exa():
		return Exa(api_key=os.getenv("EXA_API_KEY"))

## Agents

In [3]:
from textwrap import dedent
from crewai import Agent
from langchain_cohere import ChatCohere


llm = ChatCohere(model_name="command-r", temperature=0.3)


class MeetingPreparationAgents():

  def research_agent(self):
    return Agent(
        role='Research Specialist',
        goal=
        'Conduct thorough research on people and companies involved in the meeting',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Research Specialist, your mission is to uncover detailed information
          about the individuals and entities participating in the meeting. Your insights
          will lay the groundwork for strategic meeting preparation."""),
        llm=llm,
        max_rpm=5,
        verbose=True,
        memory=True)

  def industry_analysis_agent(self):
    return Agent(
        role='Tech Analyst',
        goal=
        'Analyze the current tech trends, challenges, and opportunities',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Tech Analyst, your analysis will identify key trends,
          challenges facing the tech, and potential opportunities that
          could be leveraged during the meeting for strategic advantage."""),
        llm=llm,
        max_rpm=5,
        verbose=True,
        memory=True)

  def meeting_strategy_agent(self):
    return Agent(
        role='Meeting Strategy Advisor',
        goal=
        'Develop talking points, questions, and strategic angles for the meeting',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As a Strategy Advisor, your expertise will guide the development of
          talking points, insightful questions, and strategic angles
          to ensure the meeting's objectives are achieved."""),
        llm=llm,
        max_rpm=5,
        verbose=True,
        memory=True)

  def summary_and_briefing_agent(self):
    return Agent(
        role='Briefing Coordinator',
        goal=
        'Compile all gathered information into a concise, informative briefing document',
        tools=ExaSearchTool.tools(),
        backstory=dedent("""\
          As the Briefing Coordinator, your role is to consolidate the research,
          analysis, and strategic insights."""),
        llm=llm,
        max_rpm=5,
        verbose=True,
        memory=True)


## Tasks

In [4]:
from crewai import Task

class MeetingPreparationTasks():
	def research_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Conduct comprehensive research on each of the individuals and companies
				involved in the upcoming meeting. Gather information on recent
				news, achievements, professional background, and any relevant
				business activities.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				A detailed report summarizing key findings about each participant
				and company, highlighting information that could be relevant for the meeting."""),
			async_execution=True,
			agent=agent
		)

	def industry_analysis_task(self, agent, participants, context):
		return Task(
			description=dedent(f"""\
				Analyze the current industry trends, challenges, and opportunities
				relevant to the meeting's context. Consider market reports, recent
				developments, and expert opinions to provide a comprehensive
				overview of the industry landscape.

				Participants: {participants}
				Meeting Context: {context}"""),
			expected_output=dedent("""\
				An insightful analysis that identifies major trends, potential
				challenges, and strategic opportunities."""),
			async_execution=True,
			agent=agent
		)

	def meeting_strategy_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Develop strategic talking points, questions, and discussion angles
				for the meeting based on the research and industry analysis conducted

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				Complete report with a list of key talking points, strategic questions
				to ask to help achieve the meetings objective during the meeting."""),
			agent=agent
		)

	def summary_and_briefing_task(self, agent, context, objective):
		return Task(
			description=dedent(f"""\
				Compile all the research findings, industry analysis, and strategic
				talking points into a concise, comprehensive briefing document for
				the meeting.
				Ensure the briefing is easy to digest and equips the meeting
				participants with all necessary information and strategies.

				Meeting Context: {context}
				Meeting Objective: {objective}"""),
			expected_output=dedent("""\
				A well-structured briefing document that includes sections for
				participant bios, industry overview, talking points, and
				strategic recommendations."""),
			agent=agent
		)

## Crew

In [5]:
from crewai import Crew


tasks = MeetingPreparationTasks()
agents = MeetingPreparationAgents()

participants = "Andrew Ng, Andrej Karpathy, Demis Hassabis"
context = "Comptetitive analysis on AI Agents topic"
objective = "Convince participants that the company should purchase Agents of Tomorrow, which is a startup that created a promising framework for orchestrating AI Agents"


In [6]:
# Create Agents
researcher_agent = agents.research_agent()
industry_analyst_agent = agents.industry_analysis_agent()
meeting_strategy_agent = agents.meeting_strategy_agent()
summary_and_briefing_agent = agents.summary_and_briefing_agent()

# Create Tasks
research = tasks.research_task(researcher_agent, participants, context)
industry_analysis = tasks.industry_analysis_task(industry_analyst_agent, participants, context)
meeting_strategy = tasks.meeting_strategy_task(meeting_strategy_agent, context, objective)
summary_and_briefing = tasks.summary_and_briefing_task(summary_and_briefing_agent, context, objective)

meeting_strategy.context = [research, industry_analysis]
summary_and_briefing.context = [research, industry_analysis, meeting_strategy]


In [7]:


# Create Crew responsible for Copy
crew = Crew(
	agents=[
		researcher_agent,
		industry_analyst_agent,
		meeting_strategy_agent,
		summary_and_briefing_agent
	],
	tasks=[
		research,
		industry_analysis,
		meeting_strategy,
		summary_and_briefing
	]
)



In [8]:
result = crew.kickoff()


# Print results
print("\n\n################################################")
print("## Here is the result")
print("################################################\n")
print(result)



> Entering new CrewAgentExecutor chain...


> Entering new CrewAgentExecutor chain...
Action: search, find_similar, get_contents
Action Input: {'query': 'Andrew Ng'}
Observation: I found some information on Andrew Ng:
- Andrew Ng is the founder and CEO of deeplearning.ai, which provides online deep learning courses. He is also the former Chief Scientist at Baidu, where he led the company’s Artificial Intelligence Group and founded several AI initiatives.
- He has taught machine learning courses at Stanford University and is currently an Adjunct Professor at the University of California, Berkeley.
- Ng's research interests lie in deep learning and its applications, with a focus on making the technology more accessible. He has authored several books and publications on the subject.

I also found some news about Andrew Ng: 
- Andrew Ng announced a new AI transformation program called AI Transform to help businesses adopt artificial intelligence. The program aims to help companies identi

In [9]:
from IPython.display import display, Markdown
display(Markdown(result))

## Participant Bios

- Andrew Ng is a renowned AI pioneer and entrepreneur. As the founder of deeplearning.ai, he offers specialized courses in deep learning. His practical and academic contributions include the role of Chief Scientist at Baidu, where he led the company's AI initiatives, and his current position as an Adjunct Professor at the University of California, Berkeley. Ng's latest endeavor, AI Transform, aims to guide businesses in their AI adoption journey.

- Andrej Karpathy, the Chief Technical Officer of Tesla, possesses a captivating vision for the role of AI in the automotive industry. His work at Tesla is focused on enhancing the company's products and services through cutting-edge AI development. Karpathy believes that AI has the potential to revolutionize not only the automotive sector but also robotics and human-computer interaction.

- Demis Hassabis, the founder and CEO of DeepMind, is a true AI visionary. His goal is to harness the power of AI for the betterment of society and solve complex global challenges. DeepMind has achieved international acclaim, particularly with AlphaGo, an AI algorithm that made headlines by defeating a top Go player.

## Industry Overview

The world of AI Agents is rapidly evolving, with expectations of significant advancements in the near future. The growing focus on AI in the workplace holds the promise of enhanced efficiency and automation. However, along with this progress comes the challenge of negative public perception and the crucial need to address ethical concerns. 

Among the key trends identified by industry experts is the rise of 'Autonomics'—a combination of AI Agents and automation leading to more autonomous and adaptive systems. This emerging trend presents a mix of opportunities and challenges that require careful consideration and strategic planning.

## Strategic Talking Points

1. Industry Trends:
   - The future of AI Agents is full of potential, with substantial developments expected in the coming years.
   - The increasing emphasis on AI in the workplace has the potential to bring about a revolution in efficiency and automation.
   - Experts predict the emergence of Autonomics, emphasizing the integration of AI Agents and automation for enhanced system intelligence.

2. Challenges:
   - Negative public perception remains a significant hurdle in the widespread adoption of AI Agents.
   - As AI continues to advance, addressing ethical considerations is vital for ensuring responsible development and acceptance.

3. Opportunities:
   - Educational initiatives can play a crucial role in improving public perception and showcasing the benefits of AI Agents.
   - The current industry landscape offers opportunities for innovative AI solutions and startups to emerge and thrive.
   - Developing countries' willingness to adopt new technologies could be a fertile ground for implementing and testing AI Agents.

4. Key Industry Insights:
   - Andrej Karpathy's work showcases the transformative potential of AI in the automotive industry.
   - Demis Hassabis and the accomplishments of DeepMind highlight the potential of AI systems that learn from human feedback, focusing on complex problem-solving.

5. Meeting Focus:
   - Emphasize the immense potential of AI Agents to transform industries and discuss their future impact.
   - Highlight the importance of addressing public perception and ethical concerns for successful integration.
   - Explore the advantages of Autonomics and strategic possibilities for the company's growth.

## Strategic Questions

1. How can we effectively counter negative public perception and showcase the advantages of AI Agents?
2. What are the key ethical considerations we must prioritize when implementing AI Agents to ensure responsible and sustainable development?
3. To stay at the forefront of the emerging Autonomics trend, what proactive strategies should we employ?
4. Do we have any internal insights or experiences regarding AI Agents and their potential applications that can contribute to our strategy?
5. What value would acquiring Agents of Tomorrow add to our company, and how should we evaluate this opportunity?

## Discussion Angles

1. Delve into the future prospects of AI Agents, with a special focus on the role of Autonomics in shaping the technological landscape.
2. Explore the significance of public perception and its impact on the success of AI technologies, discussing strategies to manage it effectively.
3. Examine ethical dilemmas and learn from the experiences of other companies that have successfully tackled similar challenges.
4. Discuss the role of the startup ecosystem in fostering innovative AI solutions, specifically highlighting the potential of Agents of Tomorrow.
5. Gain insights from the trajectories of Tesla and DeepMind's success and leadership in the AI domain, and explore their visions for the future. 

Is there any further information required or specific angles you wish to explore to refine your meeting strategy? This briefing document presents a comprehensive overview, and with the gathered information, you are well-equipped to make informed decisions.